In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf
from sklearn.utils import shuffle
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow.keras import models, layers
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import Model

In [2]:
import pathlib
data_dir =pathlib.Path("dataset")

In [3]:
data_dir

WindowsPath('dataset')

In [4]:
retina_images_dict = {
    'mild' : list(data_dir.glob('Mild/*')),
    'moderate' : list(data_dir.glob('Moderate/*')),
    'no_dr' : list(data_dir.glob('No_DR/*')),
    'proliferate_dr' : list(data_dir.glob('Proliferate_DR/*')),
    'severe' : list(data_dir.glob('Severe/*')),
}

In [5]:
retina_labels_dict = {
    'mild' : 0,
    'moderate' : 1,
    'no_dr' : 2,
    'proliferate_dr' : 3,
    'severe' : 4
}

In [6]:
x, y = [], []
for disease_level, images in retina_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        x.append(img)
        y.append(retina_labels_dict[disease_level])

In [7]:
x = np.array(x)
y = np.array(y)

In [8]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=879)

In [9]:
xval, xtest, yval, ytest = train_test_split(xtest, ytest, test_size=0.5, random_state=879)

In [10]:
img_size = 224
b_size = 32
channels = 3
epoch = 2

In [11]:
#scaling
resize_rescale = tf.keras.Sequential([
    #all the image have the same size then why to use??
    #when user input new image of different size then useful
    layers.experimental.preprocessing.Resizing(img_size, img_size),
    layers.experimental.preprocessing.Rescaling(1.0/255),
])

In [12]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [13]:
pre_trained_model = InceptionV3(input_shape =(img_size, img_size, channels),
                               include_top=False,
                               weights='imagenet')

In [14]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(pre_trained_model.output)
x = layers.Dense(15000, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(5000, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1000, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(5, activation="softmax")(x)

In [ ]:
model = Model(pre_trained_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001),
             loss =tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
             metrics = ['accuracy'])

In [ ]:
history = model.fit_generator(
    xtrain, ytrain,
    validation_data = (xval, yval),
    epochs = epoch,
    verbose = 2,
    batch_size = b_size
)